# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df=pd.read_csv("../WeatherPy/cities.csv")

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
fig=gmaps.figure()

locations=cities_df[["Latitude", "Longitude"]]

humidity=cities_df["Humidity"]

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

fig.add_layer(heatmap_layer)

In [4]:
fig

Figure(layout=FigureLayout(height='420px'))

In [26]:
cities_df.head()

,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed (mph)
0,Camacha,PT,33.08,-16.33,68.00,88,75,12.75
1,Mataura,NZ,-46.19,168.86,42.96,94,100,7.09
2,Hobart,AU,-42.88,147.33,44.60,81,20,5.82
3,Manacor,ES,39.57,3.21,73.40,83,0,2.24
4,São José da Coroa Grande,BR,-8.90,-35.15,75.54,82,2,6.69


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#find all cities with a temp between (75 and 90), humidity less than 75 percent, and less than 70 percent clouds.

desired_temp=cities_df.loc[cities_df["Temperature"] < 88]
desired_temp=desired_temp.loc[desired_temp["Temperature"] > 80]

desired_humidity=desired_temp.loc[desired_temp["Humidity"] < 65]


hotel_df=desired_humidity.loc[desired_humidity["Cloudiness"] < 50]


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df["Hotel Name"]=""

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
#https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=-33.8670522,151.1957362&radius=1500&type=restaurant&keyword=cruise&key=YOUR_API_KEY


params={"radius": 5000,
        "types": "lodging",
        "keyword": "hotel",
        "key": g_key}

for index, row in hotel_df.iterrows():
    
    lat=row["Latitude"]
    lng=row["Longitude"]
    
    params["location"]=f"{lat},{lng}"
    
    url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response=requests.get(url, params=params)
    
    json_response=response.json()
    
    #pprint(json_response)
    try:
        hotel_df.loc[index, "Hotel Name"]=json_response["results"][0]["name"]
    
    except (KeyError, IndexError):
        print("Skipped location.")
    
    
    

Skipped location.
Skipped location.
Skipped location.
Skipped location.
Skipped location.


In [21]:
nan_value = float("NaN")
hotel_df["Hotel Name"].replace("", nan_value, inplace=True)
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)
hotel_df

/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Country,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed (mph),Hotel Name
50,Tiarei,PF,-17.53,-149.33,84.20,51,20,11.41,Le Rocher de Tahiti
70,Saint George,US,37.10,-113.58,84.00,12,20,16.11,Best Western Plus Abbey Inn
83,Half Moon Bay,US,37.46,-122.43,82.00,34,5,12.75,"The Ritz-Carlton, Half Moon Bay"
156,Don Benito,ES,38.96,-5.86,80.01,48,0,5.01,Hotel Vegas Altas | Don Benito
167,Lumby,CA,50.25,-118.97,80.60,32,1,1.88,Twin Creeks Motel
189,Livermore,US,37.68,-121.77,84.20,26,1,17.22,Hilton Garden Inn Livermore
194,Sabha,LY,27.04,14.43,83.43,21,0,9.78,فندق المواجيد
255,Pinawa,CA,50.15,-95.88,86.72,48,2,5.61,Pinawa Motel
257,Riyadh,SA,24.69,46.72,87.80,22,0,4.70,Hyatt Regency Riyadh Olaya
264,Ciudad Real,ES,39.00,-4.00,84.99,34,0,0.83,Hotel Pago del Vicario


In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [23]:
figure_layout = {
    'width': '800px',
   'height': '600px',
    'border': '1px solid black',
   'padding': '1px',
   'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [24]:
# Add marker layer ontop of heat map
marker_layer=gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(heatmap_layer)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…